In [6]:
import pandas as pd

def calculate_t_profit(buy_price, shares, 
                       comm_rate=0.00025,  # 佣金 万2.5
                       stamp_duty=0.0005,  # 印花税 万5 (卖出收)
                       transfer_fee=0.00001, # 过户费 十万1 (双向)
                       min_comm=5.0):      # 最低佣金 5元
    
    # --- 1. 计算买入成本 ---
    buy_amt = buy_price * shares
    # 佣金：按比例算，不足5元按5元收
    buy_comm = max(buy_amt * comm_rate, min_comm)
    # 过户费
    buy_transfer = buy_amt * transfer_fee
    
    # 实际支付出的总金额 (本金 + 手续费)
    total_buy_outlay = buy_amt + buy_comm + buy_transfer
    
    print(f"【买入详情】 价格: {buy_price}, 数量: {shares}")
    print(f"买入费用: 佣金 {buy_comm:.2f} + 过户费 {buy_transfer:.2f} = {(buy_comm+buy_transfer):.2f}")
    print("-" * 30)

    # --- 2. 寻找最低获利点 (保本价) ---
    # 从买入价开始，每次加0.01，直到净利润 > 0
    break_even_price = buy_price
    found = False
    
    while not found:
        sell_amt = break_even_price * shares
        
        # 卖出各项费用
        sell_comm = max(sell_amt * comm_rate, min_comm)
        sell_transfer = sell_amt * transfer_fee
        sell_tax = sell_amt * stamp_duty # 印花税
        
        # 卖出到手金额
        net_proceeds = sell_amt - sell_comm - sell_transfer - sell_tax
        
        if net_proceeds > total_buy_outlay:
            found = True
        else:
            break_even_price += 0.01
            break_even_price = round(break_even_price, 2)
            
            # 防止死循环（比如价格过低怎么都覆盖不了5元最低佣金）
            if break_even_price > buy_price * 1.5: 
                print("警告：成本过高，无法在合理范围内回本")
                break

    print(f"★ 最低保本卖出价: {break_even_price:.2f}")
    print(f"  (上涨幅度: {((break_even_price-buy_price)/buy_price)*100:.2f}%)")
    print("-" * 30)

    # --- 3. 生成 股价 vs 利润表 ---
    # 生成从 -0.5% 到 +1.5% 的价格范围
    data_list = []
    
    # 设定价格区间，步长0.01
    start_p = round(buy_price * 0.995, 2)
    end_p = round(buy_price * 1.1, 2)
    
    current_p = start_p
    while current_p <= end_p:
        sell_amt = current_p * shares
        sell_comm = max(sell_amt * comm_rate, min_comm)
        sell_transfer = sell_amt * transfer_fee
        sell_tax = sell_amt * stamp_duty
        
        total_trade_cost = (buy_comm + buy_transfer) + (sell_comm + sell_transfer + sell_tax)
        net_profit = (sell_amt - sell_comm - sell_transfer - sell_tax) - total_buy_outlay
        
        data_list.append({
            "卖出价格": current_p,
            "涨幅(%)": round((current_p - buy_price) / buy_price * 100, 2),
            "交易总成本": round(total_trade_cost, 2),
            "净利润": round(net_profit, 2)
        })
        current_p += 0.01
        current_p = round(current_p, 2)
        
    df = pd.DataFrame(data_list)
    return df

# --- 使用示例 ---
# 设置：买入价 10.00元， 买入 2000股 (2万元)
buy_price_input = 5.24
shares_input = 200 

df_result = calculate_t_profit(buy_price_input, shares_input)
print("\n利润表预览 (部分):")
print(df_result.to_markdown(index=False))


【买入详情】 价格: 5.24, 数量: 200
买入费用: 佣金 5.00 + 过户费 0.01 = 5.01
------------------------------
★ 最低保本卖出价: 5.30
  (上涨幅度: 1.15%)
------------------------------

利润表预览 (部分):
|   卖出价格 |   涨幅(%) |   交易总成本 |   净利润 |
|-----------:|----------:|-------------:|---------:|
|       5.21 |     -0.57 |        10.54 |   -16.54 |
|       5.22 |     -0.38 |        10.54 |   -14.54 |
|       5.23 |     -0.19 |        10.54 |   -12.54 |
|       5.24 |      0    |        10.54 |   -10.54 |
|       5.25 |      0.19 |        10.55 |    -8.55 |
|       5.26 |      0.38 |        10.55 |    -6.55 |
|       5.27 |      0.57 |        10.55 |    -4.55 |
|       5.28 |      0.76 |        10.55 |    -2.55 |
|       5.29 |      0.95 |        10.55 |    -0.55 |
|       5.3  |      1.15 |        10.55 |     1.45 |
|       5.31 |      1.34 |        10.55 |     3.45 |
|       5.32 |      1.53 |        10.55 |     5.45 |
|       5.33 |      1.72 |        10.55 |     7.45 |
|       5.34 |      1.91 |        10.56 |     9.44 |
| 